In [1]:
import requests
import bs4
from lxml import etree
import pandas as pd
import re, json
from user_agent import generate_user_agent

In [12]:

# main_link = "https://market.yandex.ru/catalog--mobilnye-telefony/54726/list"
# phones_sel = "#subcategoryList > div.product_category_list > div >"
#     " div > div.subcategory-product-item__body > "
#     "div.product_name.cms_item_panel.subcategory-product-item__info > span > a"

phones_sel = "div.product_name.cms_item_panel.subcategory-product-item__info > span > a"

links = []
for p in range(1,39):
    main_link = f"https://www.citilink.ru/catalog/mobile/cell_phones/?available=1&status=55395790&p={p}"
    req = requests.get(main_link)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    for elm in parser.select(phones_sel):
        links.append(elm["href"])

In [15]:
with open("./citilink_links.json", "w") as f:
    json.dump(links, f)
    

In [21]:
def parse_for_reviews(text):
    parser = bs4.BeautifulSoup(text, 'lxml')
    for review in parser.select("div.opinion"):
        adv = review.select_one("article > ul:nth-child(2)")
        disadv = review.select_one("article > ul:nth-child(4)")
        comments = review.findAll("p")
        rating=5
        for idx, star in enumerate(review.select("div.ratings > span")):
            if "selected" in star["class"]:
                break
            rating-=1
        yield {
            "advatages": adv.get_text() if adv else "",
            "disadvatages": disadv.get_text() if disadv else "",
            "comment": ""+"\n".join([e.get_text() for e in comments]),
            "rating": rating
        }

In [22]:
results = []
for link in set(links):
    review_link = link+"getall-opinion/"
    s = requests.Session()
    headers = {
        "Host": "www.citilink.ru",
        "Connection": "keep-alive",
        "X-Requested-With": "XMLHttpRequest",
        'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))
    }
    req = requests.post(review_link, headers=headers, timeout=11)
    response_obj = json.loads(req.text)
    results.extend(list(parse_for_reviews(response_obj["html"])))

In [24]:
with open("./citilink_reviews.json", "w") as f:
    json.dump(results, f)
    

In [25]:
len(results)

1652